In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns

matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
    }
)

In [ ]:
def _compute_errors(model_evaluation):
    model_evaluation[absolute_prediction_error] = model_evaluation["Percentage Full [%%]"] - model_evaluation["Predictions"]
    model_evaluation[relative_prediction_error] = model_evaluation[absolute_prediction_error] / model_evaluation["Percentage Full [%%]"] * 100

In [ ]:
gaps = []

model_folder = None
plot_scenario += 1

if plot_scenario == 1:
    model_folder = "DEFINE"
    gaps= []
    scenario = "Scenario 1 %s"
    
if plot_scenario == 2:
    model_folder = "DEFINE"
    gaps = [
        (0.575, 0.625), # DEFINE, or REMOVE
        (0.675, 0.8) # DEFINE, or REMOVE
        ]
    scenario = "Scenario 2 %s"
    
if plot_scenario == 3:
    model_folder = "DEFINE"
    gaps = [
        (0,.62), # DEFINE, or REMOVE
        (0.81,1) # DEFINE, or REMOVE
        ]
    scenario = "Scenario 3 %s"

if plot_scenario == 4:
    model_folder = "DEFINE"
    gaps = [
        (.62,.81) # DEFINE, or REMOVE
        ]
    scenario = "Scenario 4 %s"
    
positions = ["pos3"]

model_performance = list()
best_model_mean = None
best_model_range = None
best_absolute_mean = None

best_absolute_quantile_range = None

evaluation_key = "_evaluation_set_result.csv"

for current_experiment in positions:
    use_best_mean = True

    absolute_prediction_error = "Absolute Prediction Error [mm]"
    relative_prediction_error = "Relative Prediction Error [%]"

    all_models = list()

    for file in os.listdir(model_folder):

        if not current_experiment in file:
            continue

        if evaluation_key in file:

            model_name_end_index = file.index(evaluation_key)

            model_name = file[0:model_name_end_index]
            all_models.append(model_name)


    for model in all_models:
        
        if not "m_4_conv_5" in model:
            continue

        evaluation_path = model_folder + model + evaluation_key
        evaluation = pd.read_csv(evaluation_path)

        _compute_errors(evaluation)

        mean = evaluation[relative_prediction_error].pow(2).apply(np.sqrt).mean()
        model_performance.append((mean, model))
        absolute_mean = abs(mean)

        quantiles = evaluation[relative_prediction_error].quantile([.05,.5,.95])

        quantile_range = abs(quantiles[0.95] - quantiles[0.05])

        print("%s mean error: %.5f" % (model, mean))
        print("%s quantile range: %.5f\n" % (model, quantile_range))

        if best_absolute_mean is None or absolute_mean < best_absolute_mean:
            best_absolute_mean = absolute_mean
            best_model_mean = model

        if best_absolute_quantile_range is None or quantile_range < best_absolute_quantile_range:
            best_absolute_quantile_range = quantile_range
            best_model_range = model

    print("Best model (mean) is: %s with mean error: %f" % (best_model_mean, best_absolute_mean))
    print("Best model (quantile range) is: %s with mean error: %f" % (best_model_range, best_absolute_quantile_range))

In [ ]:
performance = np.array(model_performance)
performance = pd.DataFrame({"Name" : performance[:,1], "Error": performance[:,0]})
performance = performance.astype({"Error" : "float"})
performance.sort_values(by="Error")

In [ ]:
model_name = None

if use_best_mean:
    model_name = best_model_mean
else:
    model_name = best_model_range

evaluation_path = model_folder + model_name + evaluation_key

In [ ]:
evaluation = pd.read_csv(evaluation_path)
_compute_errors(evaluation)

print("Model: '%s'" % model_name)

evaluation

In [ ]:
evaluation = evaluation.sort_values(by="Percentage Full [%%]", ignore_index=True)

In [ ]:
lower_quantile = .05
upper_quantile = .95

In [ ]:
absolute_error_quantiles = evaluation[absolute_prediction_error].quantile([lower_quantile, upper_quantile])
absolute_error_quantiles

In [ ]:
relative_error_quantiles = evaluation[relative_prediction_error].quantile([lower_quantile, upper_quantile])
relative_error_quantiles

In [ ]:
rrmse = evaluation[relative_prediction_error].pow(2).apply(np.sqrt).mean()
rrmse

In [ ]:
print("Gap Analysis")

rolling_window_size = 50

figure, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation["Percentage Full [%%]"].plot(ax=axes[0], label="Measurements", title="Measurements vs. Predictions")
evaluation["Predictions"].plot(ax=axes[0])

axes[0].legend()

evaluation[absolute_prediction_error].plot(ax=axes[1], title=absolute_prediction_error)
evaluation[absolute_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[1], title=absolute_prediction_error)

axes[1].axhline(evaluation[absolute_prediction_error].mean(), color="green")
axes[1].axhline(absolute_error_quantiles[lower_quantile], color="red")
axes[1].axhline(absolute_error_quantiles[upper_quantile], color="red")

axes[1].axhline(min(evaluation[absolute_prediction_error]), color="black")
axes[1].axhline(max(evaluation[absolute_prediction_error]), color="black")

evaluation[relative_prediction_error].plot(ax=axes[2], title=relative_prediction_error)
evaluation[relative_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[2], title=relative_prediction_error)

axes[2].axhline(evaluation[relative_prediction_error].mean(), color="green")
axes[2].axhline(relative_error_quantiles[lower_quantile], color="red")
axes[2].axhline(relative_error_quantiles[upper_quantile], color="red")

axes[2].axhline(min(evaluation[relative_prediction_error]), color="black")
axes[2].axhline(max(evaluation[relative_prediction_error]), color="black")

lower_gap = .7
upper_gap = .8

gap_start_x = np.argmax(evaluation["Percentage Full [%%]"] >= lower_gap)
gap_end_x = np.argmax(evaluation["Percentage Full [%%]"] >= upper_gap)

markers = list()

for i in range(11):
    
    current_percentage = lower_gap + ((i) / 100)
    marker = np.argmax(evaluation["Percentage Full [%%]"] >= current_percentage)
    markers.append(marker)
    
    axes[2].axvline(marker, color="black", linestyle="--")
    
axes[0].axhline(lower_gap, color="black", linestyle="--")
axes[0].axhline(upper_gap, color="black", linestyle="--")

gap_size = (upper_gap - lower_gap) * 100
axes[2].axhline(gap_size, color="black", linestyle="--")
axes[2].axhline(-gap_size, color="black", linestyle="--")


n = len(evaluation)
gap_start_scaled = gap_start_x / n
gap_end_scaled = gap_end_x / n

axes[2].axhline(gap_size/2, color="green", linestyle="--")
axes[2].axhline(-gap_size/2, color="green", linestyle="--")

for ax in axes:
    ax.axvline(gap_start_x, color="black", linestyle="--")
    ax.axvline(gap_end_x, color="black", linestyle="--")
        
plt.show()
plt.close()


fig = plt.figure(figsize=(15, 15/1.6))

start_x = int(0.9 * gap_start_x)
end_x = int(1.1 * gap_end_x)


plt.plot(evaluation[relative_prediction_error].iloc[start_x : end_x])
plt.plot(evaluation[relative_prediction_error].iloc[start_x : end_x].rolling(rolling_window_size).mean())

plt.hlines(0, xmin=gap_start_x, xmax=gap_end_x, color="red")
plt.hlines(y=[-5,5], xmin=gap_start_x, xmax=gap_end_x, color="green", linestyle="--")
plt.hlines(y=[-10,10], xmin=gap_start_x, xmax=gap_end_x, color="red", linestyle="--")


percentage = 70
for marker in markers:
    plt.vlines(marker,ymin=-15, ymax=15, color="black", linestyle="--")
    plt.text(marker-15, -16.0, str(percentage))
    percentage += 1
plt.show()
plt.close()


In [ ]:
print("Intermittent Analysis")

rolling_window_size = 50

figure, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation["Percentage Full [%%]"].plot(ax=axes[0], label="Measurements", title="Measurements vs. Predictions")
evaluation["Predictions"].plot(ax=axes[0])
evaluation["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes[0])

axes[0].legend()

evaluation[absolute_prediction_error].plot(ax=axes[1], title=absolute_prediction_error)
evaluation[absolute_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[1], title=absolute_prediction_error)

axes[1].axhline(evaluation[absolute_prediction_error].mean(), color="green")
axes[1].axhline(absolute_error_quantiles[lower_quantile], color="red")
axes[1].axhline(absolute_error_quantiles[upper_quantile], color="red")

axes[1].axhline(min(evaluation[absolute_prediction_error]), color="black")
axes[1].axhline(max(evaluation[absolute_prediction_error]), color="black")

evaluation[relative_prediction_error].plot(ax=axes[2], title=relative_prediction_error)
evaluation[relative_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[2], title=relative_prediction_error)

axes[2].axhline(evaluation[relative_prediction_error].mean(), color="green")
axes[2].axhline(relative_error_quantiles[lower_quantile], color="red")
axes[2].axhline(relative_error_quantiles[upper_quantile], color="red")

axes[2].axhline(min(evaluation[relative_prediction_error]), color="black")
axes[2].axhline(max(evaluation[relative_prediction_error]), color="black")

markers = list()
    
for axis in axes:
    for gap in gaps:
        gap_lower = np.argmax(evaluation["Percentage Full [%%]"] >= gap[0])
        gap_upper = np.argmax(evaluation["Percentage Full [%%]"] >= gap[1])
        axis.axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")

for i in [85]:
    current_percentage = i / 100.
    marker = np.argmax(evaluation["Percentage Full [%%]"] >= current_percentage)
    markers.append(marker)
    
    for axis in axes:
        axis.axvline(marker, color="black", linestyle="--")
        if 60 <= i < 100:
            plt.text(marker-50, -20, str(i))

        
plt.show()
plt.close()

In [ ]:
rolling_window_size = 50

figure, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation["Percentage Full [%%]"].plot(ax=axes[0], label="Measurements", title="Measurements vs. Predictions")
evaluation["Predictions"].plot(ax=axes[0], alpha=.6)
evaluation["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes[0])

axes[0].legend(["Measurements", "Predictions", "Predictions (Rolling Average)"])
axes[0].set_ylabel("Percentage Full [%]", fontsize="x-large")

evaluation[relative_prediction_error].plot(ax=axes[1], title=relative_prediction_error)
evaluation[relative_prediction_error].rolling(rolling_window_size).mean().plot(ax=axes[1], title=relative_prediction_error)

axes[1].axhline(evaluation[relative_prediction_error].mean(), color="green")
axes[1].axhline(relative_error_quantiles[lower_quantile], color="red")
axes[1].axhline(relative_error_quantiles[upper_quantile], color="red")

axes[1].axhline(min(evaluation[relative_prediction_error]), color="black")
axes[1].axhline(max(evaluation[relative_prediction_error]), color="black")

markers = list()

axes[1].set_ylabel("Relative Error [%%]", fontsize="x-large")



    
for axis in axes:
    for gap in gaps:
        gap_lower = np.argmax(evaluation["Percentage Full [%%]"] >= gap[0])
        gap_upper = np.argmax(evaluation["Percentage Full [%%]"] >= gap[1])
        axis.axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")

plt.show()
plt.close()

In [ ]:
rolling_window_size = 50

figure, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation["Predictions"].plot(ax=axes, alpha=.6, color="#abd9e9")
evaluation["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes, color="#2c7bb6")
evaluation["Percentage Full [%%]"].plot(ax=axes, label="Measurements", color="#fdae61")

axes.legend(["Predictions", "Predictions (Rolling Average)", "Measurements"])
axes.set_ylabel("Percentage Full [%]", fontsize="x-large")
axes.set_xlabel("# Measurements", fontsize="x-large")



for gap in gaps:
    gap_lower = np.argmax(evaluation["Percentage Full [%%]"] >= gap[0])
    gap_upper = np.argmax(evaluation["Percentage Full [%%]"] >= gap[1])
    axes.axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")


plt.show()
plt.close()

In [ ]:
rolling_window_size = 100

figure, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

evaluation["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes, alpha=.6, color="#abd9e9")
evaluation["Predictions"].rolling(rolling_window_size).mean().plot(ax=axes, color="#2c7bb6")
evaluation["Percentage Full [%%]"].rolling(rolling_window_size).mean().plot(ax=axes, label="Measurements", color="#fdae61")

axes.legend(["Predictions (Rolling Average)", "Measurements"])
axes.set_ylabel("Percentage Full [%]", fontsize="x-large")
axes.set_xlabel("# Measurements", fontsize="x-large")



for gap in gaps:
    gap_lower = np.argmax(evaluation["Percentage Full [%%]"] >= gap[0])
    gap_upper = np.argmax(evaluation["Percentage Full [%%]"] >= gap[1])
    axes.axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")


plt.show()
plt.close()

In [ ]:
print("Mean relative prediction error: %.2f" % evaluation[relative_prediction_error].mean())
print("Mean relative prediction error 5th percentile: %.2f%%" % relative_error_quantiles[lower_quantile])
print("Mean relative prediction error 95th percentile: %.2f%%" % relative_error_quantiles[upper_quantile])

In [ ]:
number_of_bins = 21 # 5% steps
bin_percentages = np.linspace(0,100,21)
bin_labels = ["L%d" % i for i in bin_percentages]
percentage_label = "Percentage Label"

labels, bins = pd.cut(evaluation["Percentage Full [%%]"], number_of_bins, labels=bin_labels, retbins=True)
evaluation[percentage_label] = labels

group_means = evaluation.groupby(by=percentage_label).mean()
group_quantiles = evaluation[[percentage_label, relative_prediction_error]].groupby(by=percentage_label).quantile([.05, .95])

group_quantiles.index.set_names(["Label", "Quantile"], inplace=True)

upper_quantiles = group_quantiles.iloc[group_quantiles.index.get_level_values('Quantile') == upper_quantile]
lower_quantiles = group_quantiles.iloc[group_quantiles.index.get_level_values('Quantile') == lower_quantile]

lower = lower_quantiles["Relative Prediction Error [%]"].values
upper = upper_quantiles["Relative Prediction Error [%]"].values
means = group_means["Relative Prediction Error [%]"].values


In [ ]:
fig = plt.figure(figsize=(10, 10/1.6))
fig.tight_layout()

zero_line_y = np.zeros(len(bin_labels))

plt.plot(bin_labels, lower)
plt.plot(bin_labels, upper)
plt.plot(bin_labels, means)


plt.plot(bin_labels, zero_line_y, alpha=0)
plt.fill_between(bin_labels, lower, upper, facecolor="C0", alpha=0.05)
plt.fill_between(bin_labels, zero_line_y, means, where=(zero_line_y < means), facecolor="C1", alpha=0.35)
plt.fill_between(bin_labels, zero_line_y, means, where=(zero_line_y > means), facecolor="C2", alpha=0.35)


plt.show()
plt.close()

In [ ]:
factor = 10

new_bin_indexes = np.linspace(0, 100, number_of_bins * factor)

lower_inderpolated = np.interp(new_bin_indexes, bin_percentages, lower)
upper_inderpolated = np.interp(new_bin_indexes, bin_percentages, upper)
means_inderpolated = np.interp(new_bin_indexes, bin_percentages, means)

In [ ]:
fig = plt.figure(figsize=(10, 10/1.6))
fig.tight_layout()

zero_line_y = np.zeros(len(lower_inderpolated))

plt.plot(new_bin_indexes, lower_inderpolated)
plt.plot(new_bin_indexes, upper_inderpolated)
plt.plot(new_bin_indexes, means_inderpolated)

plt.ylabel("Relative Error (Expected-Prediction) [%%]")
plt.xlabel("Percentage Full [%]")

plt.plot(new_bin_indexes, zero_line_y, alpha=0)
plt.fill_between(new_bin_indexes, lower_inderpolated, upper_inderpolated, facecolor="C0", alpha=0.05)
plt.fill_between(new_bin_indexes, zero_line_y, means_inderpolated, where=(zero_line_y < means_inderpolated), facecolor="C1", alpha=0.35)
plt.fill_between(new_bin_indexes, zero_line_y, means_inderpolated, where=(zero_line_y > means_inderpolated), facecolor="C2", alpha=0.35)

plt.xticks((bin_percentages))

plt.show()
plt.close()

In [ ]:
evaluation

In [ ]:
pd.set_option('display.max_rows', 558)
index = (evaluation["Percentage Full [%%]"] > .7) & (evaluation["Percentage Full [%%]"] < .8)
evaluation[index][["Percentage Full [%%]", "Predictions"]].plot()

In [ ]:
evaluation[index][["Percentage Full [%%]", "Predictions"]]

In [ ]:
table_entry = {
    "Title" : scenario,
    "Name" : model_name,
    "RRMSE" : rrmse,
    "Percentiles" : relative_error_quantiles,
    "Measurements" : evaluation["Percentage Full [%%]"],
    "Predictions" : evaluation["Predictions"],
    "Predictions_rolling" : evaluation["Predictions"].rolling(rolling_window_size).mean(),
    "Gaps" : gaps
}

In [ ]:
table_contents.append(table_entry)

In [ ]:
for t in table_contents:
    print(t["Name"])

In [ ]:
rolling_window_size = 50

n_rows = 2
n_cols = int(len(table_contents) / n_rows)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols, sharex=False, sharey=True, figsize=(15,15),constrained_layout=True)

table_content_index = 0

for i in range(n_rows):
    for j in range(n_cols):
        
        table_content = table_contents[table_content_index]
        error_string = "\n[%.2f, %.2f, %.2f]" % (table_content["Percentiles"][0.05], table_content["RRMSE"], table_content["Percentiles"][0.95])

        table_content["Predictions"].plot(ax=axes[i][j], color="#abd9e9", alpha=.6)
        table_content["Predictions_rolling"].plot(ax=axes[i][j], color="#2c7bb6")
        table_content["Measurements"].plot(ax=axes[i][j], label="Measurements", color="#fdae61")

        axes[i][j].legend(["Predictions", "Predictions (Rolling Average)", "Measurements"],loc='upper left', prop={'size': 18})
        axes[i][j].set_ylabel("Percentage Full [%]", fontsize="xx-large")

        axes[i][j].set_xlabel("Measurements", fontsize="x-large")
        axes[i][j].set_title(table_content["Title"] % error_string, fontsize="xx-large")
        axes[i][j].tick_params(axis='both', which='major', labelsize="x-large")
        axes[i][j].tick_params(axis='both', which='minor', labelsize="x-large")

        for gap in table_content["Gaps"]:
            gap_lower = np.argmax(evaluation["Percentage Full [%%]"] >= gap[0])
            gap_upper = np.argmax(evaluation["Percentage Full [%%]"] >= gap[1])
            axes[i][j].axvspan(gap_lower, gap_upper, alpha=.1, color="#d7191c")

        table_content_index += 1

plt.show()
plt.close()

In [ ]:
rolling_window_size = 50

n_rows = 2
n_cols = int(len(table_contents) / n_rows)


figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols, sharex=False, sharey=True, figsize=(15,15),constrained_layout=True)


table_content_index = 0

gap_color = "#e66101"
predictions_color = "#b2abd2"
rolling_average_color = "#5e3c99"
measurements_color = "#fdb863"

for i in range(n_rows):
    for j in range(n_cols):
            
        table_content = table_contents[table_content_index]
        
        predictions = table_content["Predictions"] * 100
        rolling_predictions = table_content["Predictions_rolling"] * 100
        measurements = table_content["Measurements"] * 100
        
        predictions.plot(ax=axes[i][j], color=predictions_color, alpha=.6)
        rolling_predictions.plot(ax=axes[i][j], color=rolling_average_color)
        measurements.plot(ax=axes[i][j], label="Measurements", color=measurements_color)

        if i == 0 and j == 0:
            axes[i][j].legend(["Predictions", "Predictions (Rolling Average)", "Measurements"],loc='upper left', prop={'size': 18})
        axes[i][j].set_ylabel("Percentage Full [%]", fontsize="xx-large")
        axes[i][j].set_xlabel("Measurements", fontsize="x-large")
        axes[i][j].set_title(table_content["Title"] % "", fontsize="xx-large")
        
        axes[i][j].tick_params(axis='both', which='major', labelsize="x-large")
        axes[i][j].tick_params(axis='both', which='minor', labelsize="x-large")
        
        axes[i][j].set_xlim(0,len(measurements))
        axes[i][j].set_ylim(40,100)
        

        for gap in table_content["Gaps"]:
            gap_lower = np.argmax(evaluation["Percentage Full [%%]"] >= gap[0])
            gap_upper = np.argmax(evaluation["Percentage Full [%%]"] >= gap[1])
            axes[i][j].axvspan(gap_lower, gap_upper, alpha=.1, color=gap_color)
        
        error_string = "Error Summary: [%.2f, %.2f, %.2f]" % (table_content["Percentiles"][0.05], table_content["RRMSE"], table_content["Percentiles"][0.95])
        error_string = \
            "Prediction Errors:\n" + \
            "5$^{th}$ percentile:\t%.2f\n" % table_content["Percentiles"][0.05] + \
            "95$^{th}$ percentile: \t%.2f\n" % table_content["Percentiles"][0.95] + \
            "RRMSE:\t\t%.2f" % table_content["RRMSE"]
         
        error_string =  f"{'RRMSE:':<25}|{table_content['RRMSE']:>10.2f}\n"  +\
                        f"{'5$^{th}$ percentile:':<25}|{table_content['Percentiles'][0.05]:>10.2f}\n" +\
                        f"{'95$^{th}$ percentile:':<25}|{table_content['Percentiles'][0.95]:>10.2f}"
        
        error_string =  f"{table_content['RRMSE']:>10.2f} {'RRMSE':>25}\n"  +\
                        f"{table_content['Percentiles'][0.05]:>10.2f}{'5$^{th}$ percentile':>32}\n" +\
                        f"{table_content['Percentiles'][0.95]:>10.2f}{'95$^{th}$ percentile':>32}"
            
        
        axes[i][j].text(6000,45,error_string, fontsize = "xx-large", linespacing=1.1)

        table_content_index += 1



plt.show()
plt.close()

In [ ]:
len(measurements)

In [ ]:
execution_blocker

In [ ]:
table_contents = list()
plot_scenario = 0